[View in Colaboratory](https://colab.research.google.com/github/dexlau/sample/blob/master/project1.ipynb)

In [0]:
import numpy as np

# Create a single random walk sequence.
def random_walk_sequence():
  # States representing A - G.
  x_t = [0., 0., 0., 1., 0., 0., 0.]
  walk = [np.array(x_t)]
  while True:
    state = x_t.index(1.)
    next_state = state+1 if np.random.randint(2) else state-1 
    x_t[next_state] = 1
    x_t[state] = 0
    walk.append(np.array(x_t))
    if next_state in [0,6]:
      return walk

# Create a set of random walk sequences.
def random_walk_sequence_set(sets):
  return [random_walk_sequence() for _ in range(sets)]

# Construct training data.
def random_walk_training_set(total_training_sets, sequences_per_set):
  return training_set = [random_walk_sequence_set(sequences_per_set) for _ in range (total_training_sets)]

# Produce prediction.
def predict(W, X):
  return np.dot(np.transpose(W), X)

In [1]:
print('hello')

hello


In [127]:
import numpy as np
import random
import matplotlib.pyplot as plt

# ============= Generate data training set==============================================
# Probability to move left or right
prob = 0.5
# Position A,B,C,D,E,F,G is defined as number 0,1,2,3,4,5,6,7
start = 3  # Start at position D

# Generate training set
TrainingSet = np.zeros((1000, 100),int)
for i in range(0, 1000):
    X = [start]
    for m in range(1, 100):
        rr = random.random()
        left = rr < prob and X[-1] > 0  # position A or number 0 is left boundary
        right = rr > prob and X[-1] < 7  # position G or number 7 is right boundary
        X.append(X[-1] - left + right)
        if X[-1] == 0 or X[-1] == 7:
            break       # end game if walk in to position A or G,
                        # meaning turn all number into 0 after the first 0 or 7
    TrainingSet[i, 0: m+1] = np.array(X)
# check random TrainingSet
#plt.plot(TrainingSet[random.randint(1, 100), :])
#plt.show()
# check all TrainingSet. make sure all 1000 games will end before 100 steps
#for i in range(0, 1000):
#    plt.plot(TrainingSet[i, :])
#    plt.show()


# ============= Reproduce Figure 3 ==============================================
# Define observation vector
X = np.identity(5, int)
XA = np.zeros((1, 5), int).T
XG = np.ones((1, 5), int).T
X = np.hstack((XA, X, XG))  # 5X7 matrix

# Set up initial value
ideal_P = np.array([[1.0/6, 1.0/3, 1.0/3, 2.0/3, 5.0/6]])
Lambda = np.array([0, 0.1, 0.3, 0.5, 0.7, 0.9, 1])
RMSE = np.array([0., 0., 0., 0., 0., 0., 0.])
alpha = 0.1
w = np.array([.5, .5, .5, .5, .5]) # start with 0.5 weights
for lam in range(0, 7):
    for m in range(0, 100):
        for n in range(0, 99):
            x = np.array([X[:, TrainingSet[m, n] - 1]]).T
            x1 = np.array([X[:, TrainingSet[m, n + 1] - 1]]).T
            delta_w = alpha * (np.dot(Lambda[lam]*w.T, x1) - np.dot(w.T, x))*x
        w = w + delta_w
    RMSE[lam] = np.sqrt(np.mean((ideal_P-w)**2))
plt.plot(Lambda, RMSE)
plt.xlabel("$lambda$")
plt.ylabel('ERROR')
plt.show()


1.25